#### Next Word Prediction Model & how to build it.

In this notebook, we will build a next word prediction model, which means predicting the most
likely word of phrase taht will come next in a sentence or text. It is like having an inbuilt 
feature on an application that suggests the next word as you type or speak. These models are 
used in many applications like messaging apps, search engines, virtual assistants, and 
autocorrect features on smartphones.

NWP is a language modelling task in Machine Learning that aims to predict the most probable word 
or sequence of words that follows a given input context. This task uses statistical patterns and 
linguistic structures to generate accurate predictions based on the context provided.

To build a NWP model:

1. Start by collecting a diverse dataset of text documents.
2. Preprocess the data by cleaning and tokenizing it.
3. Preprae the data by creating input-output pairs.
4. Engineer features such as word embeddings.
5. Select an apporpiate model like and LSTM or GPT.
6. Train the model on the dataset while adjusting hyperparameters.
7. Improve the model by experimenting with different techniques and architectures.

This iterative process allows businesses to develop accurate and efficient NWP models taht can be 
applied in diverse situations.

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file 
# Change accordingly to the file path
with open('/home/xamanek/PythonProjects/TransformersML/Datasets/20240205b_sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding = 'utf-8') as file:
    text = file.read()

In [6]:
# Tokenize the text to create a sequence of words.
tokenizer = Tokenizer()
tokenizer.fit_on_texts( [ text ] )
total_words = len( tokenizer.word_index ) + 1

The code above tokenizes the text, which means it is divided into individual words or 'tokens'.  
The 'Tokenizer' object is created, which will handle the tokenization process. The 'fit_on_texts'  
method of the tokenizer is called, passing the 'text' as input. This method analyzes the text and   
builds a vocabulary of unique words, assigning each word a numerical index. The 'total_words'  
variable is then assigned the value of the length of the word index plus one, representing the   
total number of distinct words in the text.

In [7]:
# Now we can create input-output pairs by splitting the text into 
# sequences of tokens and forming n-grams from the sequences 

input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences( [ line ] )[ 0 ]
  for i in range( 1, len( token_list ) ):
    n_gram_sequence = token_list[ : i + 1 ]
    input_sequences.append( n_gram_sequence )

The code above splits text data into lines using the '\n' character as delimiter.  
For each line in the text, the 'texts_to_sequences' method of the tokenizer is used  
to convert the line into a sequence of numerical tokens based on the previously created  
vocabulary. The resulting token list is then iterated over using a for loop. For each  
iteration, a subsequence, or n-gram, of tokens is extracted, ranging from the beginning  
of the token list up to the current index 'i'.  
  
This n-gram sequence represents the input context, with the last token being the target  
predicted word. This n-gram sequence is then appended to the 'input_sequences' list.  
This process is repeated for all lines in the text, generating multiple input-output  
sequences that will be used for training the next word prediction model.  

In [9]:
# Now lets pad the input sequences to have equal length
max_sequence_len = max( [ len(seq) for seq in input_sequences ] )
input_sequences = np.array( pad_sequences( input_sequences, maxlen = max_sequence_len, padding = 'pre' ) )

The preceding code creates the inpu sequences that are padded to ensure all sequences  
have the same length. The variable 'max_sequence_len' is assigned the maximum length  
among all the input sequences. The 'pad_sequences' function is used to pad or truncate  
the input sequences to match this maximum length.  
  
The 'pad_sequences' function takes the input_sequences list, sets the maximum length to  
'max_sequence_len', and specifies taht the padding should be added at the beginning of  
each sequence using the 'padding = pre' argument. Finally, the input sequences are  
converted into a numpy array to facilitate further processing.

In [10]:
# Now to split the sequences into input and output
X = input_sequences[ :, :-1 ]
y = input_sequences[ :, -1 ]

The input sequences are split into two arrays, 'X' and 'y', to create the input  
and output for training the next word prediction model. The 'X' array is assigned  
the values of all rows in the 'input_sequences' array except for the last column.  
It means that 'X' contains all the tokens in each sequence except for the last one,  
representing the input context.  
  
On the other hand, the 'y' array is assigned the values of the last column in the  
'input_sequences' array, which represents the target or predicted word.

In [11]:
# Convert the output to one-hot encode vectors
y = np.array(
  tf.keras.utils.to_categorical(
    y, num_classes = total_words
  )
)

In [12]:
# now to build a neural network architecture to train the model
model = Sequential()
model.add( 
  Embedding( 
    total_words, 100, input_length = max_sequence_len - 1 
  ) 
)
model.add(
  LSTM(150)
)
model.add(
  Dense(
    total_words, activation = 'softmax'
  )
)
print( model.summary() )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


2024-02-05 20:18:44.520186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-05 20:18:44.520764: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


The code above defines the model architeccture for the next word prediction model.  
The 'Sequential' model is created, which represents a linear stack of layers.  
First layer added to the model is the 'Embedding' layer, which is responsible for  
converting the input sequences into dense vectors of fixed size. It takes 3 arguments:  
  
1. 'total_words', which represents the total number of distinct words in the vocabulary.
2. '100', which denotes the dimensionality of the word embeddings.
3. and 'input_length', which specifies the length of the input sequences. 
  
The next layer added is the 'LSTM' (Long Short-Term Memory) layer, a type of recurrent neural network  
(RNN) layer designed for capturing sequential dependencies in the data. It has 150 units,  
which means it will learn 150 internal representations or memory cells.  
  
Finally, the 'Dense' layer is added, which is a fully connected layer that produces the output predictions.  
It has 'total_words' units and uses the 'softmax' activation function to convert the predicted  
scores into probabilities, indicating the likelihood of each word being the next one in  
the sequences.  

In [13]:
# Compile and train the model
model.compile(
  loss = 'categorical_crossentropy', 
  optimizer = 'adam', 
  metrics = [ 'accuracy' ]
)
model.fit(
  X, y, epochs = 100, verbose = 1 
)

Epoch 1/100


2024-02-05 20:25:50.142743: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3159099200 exceeds 10% of free system memory.


3010/3010 [==============================] - 65s 21ms/step - loss: 6.2601 - accuracy: 0.0753
Epoch 2/100
3010/3010 [==============================] - 67s 22ms/step - loss: 5.5480 - accuracy: 0.1211
Epoch 3/100
3010/3010 [==============================] - 62s 20ms/step - loss: 5.1717 - accuracy: 0.1437
Epoch 4/100
3010/3010 [==============================] - 61s 20ms/step - loss: 4.8472 - accuracy: 0.1623
Epoch 5/100
3010/3010 [==============================] - 60s 20ms/step - loss: 4.5439 - accuracy: 0.1800
Epoch 6/100
3010/3010 [==============================] - 61s 20ms/step - loss: 4.2598 - accuracy: 0.1989
Epoch 7/100
3010/3010 [==============================] - 60s 20ms/step - loss: 3.9891 - accuracy: 0.2232
Epoch 8/100
3010/3010 [==============================] - 60s 20ms/step - loss: 3.7293 - accuracy: 0.2507
Epoch 9/100
3010/3010 [==============================] - 60s 20ms/step - loss: 3.4838 - accuracy: 0.2834
Epoch 10/100
3010/3010 [==============================] - 60s 20ms/

In [18]:
# Now we can generate the next word predictions using our trained model
seed_text = "I don't know what to"
next_words = 5

for _ in range( next_words ):
  token_list = tokenizer.texts_to_sequences( [ seed_text ] )[ 0 ]
  token_list = pad_sequences( 
    [ token_list ], 
    maxlen = max_sequence_len - 1, 
    padding = 'pre' 
  )
  predicted = np.argmax( model.predict( token_list ), axis = -1 )
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word

print( seed_text )

1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 10ms/step
I don't know what to say i have a fairly
